# 지능범_절도밤_강력범_폭력범_교통범_지도시각화 

In [27]:
import mysql.connector
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt 
import koreanize_matplotlib
from plotly.subplots import make_subplots
import os, json
import pandas as pd

#### RDS DB 연결

In [28]:
aws_RDS = mysql.connector.connect(
host = "please enter your's",
user = "admin",
password = "****",
database = "database name"
)

crime = aws_RDS.cursor(dictionary=True, buffered=True)

#### 지역 별 범죄유형 별 발생 건수 중 절도범죄 데이터프레임 형성 

In [29]:
crime.execute("SELECT * FROM crime_dosi_after")
result = crime.fetchall()

crime_dosi_df = pd.DataFrame(result)
crime_dosi_df.head()

theft_dosi_df = crime_dosi_df[crime_dosi_df["중"] == "절도범죄"]
theft_dosi_df

,대,중,소,발생장소,2011년,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,2022년
935,총범죄,절도범죄,소계,서울,54310,61332,61600,59371,55280,46861,41144,39182,42224,38296,33529,37594
936,총범죄,절도범죄,소계,부산,24697,25494,23887,21898,20136,16777,15807,14373,14497,14189,12667,13699
937,총범죄,절도범죄,소계,대구,14852,16554,17481,14609,12759,9171,8310,8169,9249,8675,9306,9792
938,총범죄,절도범죄,소계,인천,10146,10394,10989,9737,10679,10025,9216,9473,10134,8991,8175,8895
939,총범죄,절도범죄,소계,광주,13373,12678,12419,10361,8438,6050,4821,4832,4922,4939,4884,5204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1018,총범죄,절도범죄,소계,통영시,0,0,0,0,602,551,480,412,466,478,412,387
1019,총범죄,절도범죄,소계,서귀포시,0,0,0,0,921,927,864,858,907,1155,860,886
1020,총범죄,절도범죄,소계,제주시,5405,5053,5834,4980,3309,2573,2359,2124,2325,2270,2194,2247
1021,총범죄,절도범죄,소계,기타도시,17440,18734,18149,17551,643,369,19,49,31,24,38,43


In [30]:
geo_path = 'korean_city.json'
geo_korea_city= json.load(open(geo_path, encoding='utf-8'))

print( geo_korea_city["cities"][0]["lat"] )
print( geo_korea_city["cities"][0]["lon"] )
print(len(geo_korea_city["cities"]))

37.5665
126.978
86


In [31]:
korea_city_lat =[]
korea_city_lon = []
exit_city = 0

for df_city in theft_dosi_df["발생장소"]:
    for city in geo_korea_city["cities"]:
        if city["name"] == df_city:    
            korea_city_lat.append(city["lat"])
            korea_city_lon.append(city["lon"])
            exit_city = 1
    
    if exit_city == 1:
        exit_city = 0
    else:
        korea_city_lat.append(0.0)
        korea_city_lon.append(0.0)

print(korea_city_lat)
print(korea_city_lon)

[37.5665, 35.1796, 35.8714, 37.4563, 35.1595, 36.3504, 35.5384, 36.48, 37.6583, 37.4322, 37.4784, 37.4294, 37.5925, 37.3616, 37.6159, 37.636, 37.9039, 37.5035, 37.4386, 37.2636, 37.3799, 37.3219, 37.0103, 37.3943, 37.7858, 37.2985, 37.1485, 37.241, 37.3455, 37.738, 37.2721, 37.7599, 36.9925, 37.8949, 37.5391, 37.1995, 37.7519, 37.5247, 37.4449, 38.207, 37.3422, 37.8813, 37.1665, 37.1323, 36.6424, 36.9911, 36.2746, 36.4469, 36.1873, 36.8899, 36.3494, 36.785, 36.79, 36.8151, 35.9676, 35.8038, 35.4164, 35.9483, 35.8242, 35.5698, 34.9406, 35.0159, 34.8124, 34.9506, 34.7604, 35.8252, 35.8562, 36.1196, 36.1398, 36.586, 36.4109, 36.5684, 36.8058, 35.9733, 36.019, 34.8804, 35.2281, 35.2089, 35.5038, 35.0038, 35.335, 35.1795, 35.228, 34.8544, 33.2539, 33.4996, 0.0, 0.0]
[126.978, 129.0756, 128.6014, 126.7052, 126.8526, 127.3845, 129.3114, 127.289, 126.832, 126.9938, 126.8663, 127.2564, 127.1296, 126.9359, 126.7156, 127.2165, 127.0607, 126.7665, 127.1378, 127.0286, 126.8032, 126.8309, 127.2632, 

In [32]:
theft_dosi_df["위도"] = korea_city_lat
theft_dosi_df["경도"] = korea_city_lon
theft_dosi_df.head()

/tmp/ipykernel_11040/3799167733.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_11040/3799167733.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,대,중,소,발생장소,2011년,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,2022년,위도,경도
935,총범죄,절도범죄,소계,서울,54310,61332,61600,59371,55280,46861,41144,39182,42224,38296,33529,37594,37.5665,126.9780
936,총범죄,절도범죄,소계,부산,24697,25494,23887,21898,20136,16777,15807,14373,14497,14189,12667,13699,35.1796,129.0756
937,총범죄,절도범죄,소계,대구,14852,16554,17481,14609,12759,9171,8310,8169,9249,8675,9306,9792,35.8714,128.6014
938,총범죄,절도범죄,소계,인천,10146,10394,10989,9737,10679,10025,9216,9473,10134,8991,8175,8895,37.4563,126.7052
939,총범죄,절도범죄,소계,광주,13373,12678,12419,10361,8438,6050,4821,4832,4922,4939,4884,5204,35.1595,126.8526


In [33]:
theft_dosi_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 88 entries, 935 to 1022
Data columns (total 18 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   대       88 non-null     object 
 1   중       88 non-null     object 
 2   소       88 non-null     object 
 3   발생장소    88 non-null     object 
 4   2011년   88 non-null     int64  
 5   2012년   88 non-null     int64  
 6   2013년   88 non-null     int64  
 7   2014년   88 non-null     int64  
 8   2015년   88 non-null     int64  
 9   2016년   88 non-null     int64  
 10  2017년   88 non-null     int64  
 11  2018년   88 non-null     int64  
 12  2019년   88 non-null     int64  
 13  2020년   88 non-null     int64  
 14  2021년   88 non-null     int64  
 15  2022년   88 non-null     int64  
 16  위도      88 non-null     float64
 17  경도      88 non-null     float64
dtypes: float64(2), int64(12), object(4)
memory usage: 13.1+ KB


In [34]:
('발생 장소: ' + theft_dosi_df["발생장소"] + ' / 발생 건수: ' + theft_dosi_df["2011년"].astype(str)).values

array(['발생 장소: 서울 / 발생 건수: 54310', '발생 장소: 부산 / 발생 건수: 24697',
       '발생 장소: 대구 / 발생 건수: 14852', '발생 장소: 인천 / 발생 건수: 10146',
       '발생 장소: 광주 / 발생 건수: 13373', '발생 장소: 대전 / 발생 건수: 12764',
       '발생 장소: 울산 / 발생 건수: 6764', '발생 장소: 세종 / 발생 건수: 0',
       '발생 장소: 고양시 / 발생 건수: 3654', '발생 장소: 과천시 / 발생 건수: 0',
       '발생 장소: 광명시 / 발생 건수: 1223', '발생 장소: 광주시 / 발생 건수: 0',
       '발생 장소: 구리시 / 발생 건수: 1222', '발생 장소: 군포시 / 발생 건수: 1088',
       '발생 장소: 김포시 / 발생 건수: 0', '발생 장소: 남양주시 / 발생 건수: 1936',
       '발생 장소: 동두천시 / 발생 건수: 0', '발생 장소: 부천시 / 발생 건수: 5783',
       '발생 장소: 성남시 / 발생 건수: 4882', '발생 장소: 수원시 / 발생 건수: 6373',
       '발생 장소: 시흥시 / 발생 건수: 1900', '발생 장소: 안산시 / 발생 건수: 3733',
       '발생 장소: 안성시 / 발생 건수: 0', '발생 장소: 안양시 / 발생 건수: 2594',
       '발생 장소: 양주시 / 발생 건수: 0', '발생 장소: 여주시 / 발생 건수: 0',
       '발생 장소: 오산시 / 발생 건수: 0', '발생 장소: 용인시 / 발생 건수: 2992',
       '발생 장소: 의왕시 / 발생 건수: 0', '발생 장소: 의정부시 / 발생 건수: 2320',
       '발생 장소: 이천시 / 발생 건수: 873', '발생 장소: 파주시 / 발생 건수: 1283',
       '발생 장소: 평택시 / 발

In [35]:
fig = go.Figure()

for  year in theft_dosi_df.columns[4:16]:
    fig.add_trace(go.Scattergeo(
        locationmode='ISO-3',
        lon=theft_dosi_df["경도"],
        lat=theft_dosi_df["위도"],
        text=('발생 장소: ' + theft_dosi_df["발생장소"] + ' / 발생 건수: ' + theft_dosi_df[year].astype(str)).values,
        name= "{0} 절도범죄 발생건수".format(year),
        marker=dict(
            size=(theft_dosi_df[year] / 500).values,
            color="crimson",
            line_color = 'rgb(40,40,40)',
            line_width = 0.5,
        ),
        mode='markers',
        visible=False
    ))

fig.data[0].visible = True

steps = []
years  = theft_dosi_df.columns[4:16]

for i in range(len(set(years))):
    step = dict(
        method = 'update',
        label=str(sorted(set(years))[i]),
        args=[{'visible':[False]*len(fig.data)},
              {'title': '{} 절도범죄 지도 시각화'.format(sorted(set(years))[i])}],
    )

    step['args'][0]['visible'][i] = True
    steps.append(step)


fig.update_layout( 
    title_text='<b>연도별 발생지역의 절도범죄 지도 시각화</b>',
    title_x=0.5, 
    title_font_size=22,
    geo=dict(
        scope='asia',
        resolution=50,
        showland=True,
        landcolor="rgb(240, 240, 240)",
        showcountries=True,
        countrycolor="rgb(170, 170, 170)",
        lataxis=dict(
            range=[33, 39]
        ),
        lonaxis=dict(
            range=[124, 131]
        ),
        projection=dict(
            type='mercator',
            scale=0.9  # Adjusting scale to make the map larger
        ),
        center=dict(
            lat=36.5,  # Centering map on South Korea
            lon=127.5
        )
    ),
    sliders=[{
        'active':0,
        'steps':steps,
    }],
    width=900,  # Adjust width of the plot
    height=1000  # Adjust height of the plot
)
fig.show()

fig.write_html("thief_dosi_map.html")

In [36]:
strong_dosi_df = crime_dosi_df[(crime_dosi_df["중"] == "강력범죄") & (crime_dosi_df["소"] == "소계") ]
strong_dosi_df

,대,중,소,발생장소,2011년,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,2022년
88,총범죄,강력범죄,소계,서울,6878,6033,6272,6268,6215,6683,7474,6974,6977,6246,5337,6250
89,총범죄,강력범죄,소계,부산,2110,1836,2203,1860,1774,1630,1936,1712,1716,1593,1481,1693
90,총범죄,강력범죄,소계,대구,1106,1196,1372,1154,1084,1028,999,1130,1095,917,925,1027
91,총범죄,강력범죄,소계,인천,1696,1522,1542,1478,1542,1817,1880,1743,1749,1524,1439,1599
92,총범죄,강력범죄,소계,광주,952,1059,978,953,860,796,803,792,753,629,587,631
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,총범죄,강력범죄,소계,통영시,0,0,0,0,58,50,48,260,58,66,48,74
172,총범죄,강력범죄,소계,서귀포시,0,0,0,0,65,128,92,110,118,103,115,124
173,총범죄,강력범죄,소계,제주시,336,348,498,398,331,365,352,322,358,347,336,326
174,총범죄,강력범죄,소계,기타도시,1633,1668,1912,1848,67,62,30,42,50,43,39,46


In [37]:
strong_dosi_df["위도"] = korea_city_lat
strong_dosi_df["경도"] = korea_city_lon
strong_dosi_df.head()

/tmp/ipykernel_11040/755249417.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_11040/755249417.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,대,중,소,발생장소,2011년,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,2022년,위도,경도
88,총범죄,강력범죄,소계,서울,6878,6033,6272,6268,6215,6683,7474,6974,6977,6246,5337,6250,37.5665,126.9780
89,총범죄,강력범죄,소계,부산,2110,1836,2203,1860,1774,1630,1936,1712,1716,1593,1481,1693,35.1796,129.0756
90,총범죄,강력범죄,소계,대구,1106,1196,1372,1154,1084,1028,999,1130,1095,917,925,1027,35.8714,128.6014
91,총범죄,강력범죄,소계,인천,1696,1522,1542,1478,1542,1817,1880,1743,1749,1524,1439,1599,37.4563,126.7052
92,총범죄,강력범죄,소계,광주,952,1059,978,953,860,796,803,792,753,629,587,631,35.1595,126.8526


In [38]:
fig = go.Figure()

for  year in strong_dosi_df.columns[4:16]:
    fig.add_trace(go.Scattergeo(
        locationmode='ISO-3',
        lon=strong_dosi_df["경도"],
        lat=strong_dosi_df["위도"],
        text=('발생 장소: ' + strong_dosi_df["발생장소"] + ' / 발생 건수: ' + strong_dosi_df[year].astype(str)).values,
        name= "{0} 강력범죄 발생건수".format(year),
        marker=dict(
            size=(strong_dosi_df[year] / 50).values,
            color="royalblue",
            line_color = 'rgb(40,40,40)',
            line_width = 0.5,
        ),
        mode='markers',
        visible=False
    ))

fig.data[0].visible = True

steps = []
years  = strong_dosi_df.columns[4:16]

for i in range(len(set(years))):
    step = dict(
        method = 'update',
        label=str(sorted(set(years))[i]),
        args=[{'visible':[False]*len(fig.data)},
              {'title': '{} 강력범죄 지도 시각화'.format(sorted(set(years))[i])}],
    )

    step['args'][0]['visible'][i] = True
    steps.append(step)


fig.update_layout( 
    title_text='<b>연도별 발생지역의 강력범죄 지도 시각화</b>',
    title_x=0.5, 
    title_font_size=22,
    geo=dict(
        scope='asia',
        resolution=50,
        showland=True,
        landcolor="rgb(240, 240, 240)",
        showcountries=True,
        countrycolor="rgb(170, 170, 170)",
        lataxis=dict(
            range=[33, 39]
        ),
        lonaxis=dict(
            range=[124, 131]
        ),
        projection=dict(
            type='mercator',
            scale=0.9  # Adjusting scale to make the map larger
        ),
        center=dict(
            lat=36.5,  # Centering map on South Korea
            lon=127.5
        )
    ),
    sliders=[{
        'active':0,
        'steps':steps,
    }],
    width=900,  # Adjust width of the plot
    height=1000  # Adjust height of the plot
)
fig.show()

fig.write_html("strong_dosi_map.html")

In [39]:
mastermind_dosi_df = crime_dosi_df[(crime_dosi_df["중"] == "지능범죄") & (crime_dosi_df["소"] == "소계") ]
mastermind_dosi_df

,대,중,소,발생장소,2011년,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,2022년
1815,총범죄,지능범죄,소계,서울,71386,72333,76756,72052,72851,72137,70294,74668,78857,82443,64838,72239
1816,총범죄,지능범죄,소계,부산,28310,25502,28191,24263,25247,25052,23726,27672,31357,36784,30249,32816
1817,총범죄,지능범죄,소계,대구,12957,12859,13639,12868,14195,13227,13206,15021,18218,19812,17979,18192
1818,총범죄,지능범죄,소계,인천,12511,14059,18957,16580,16938,17248,16147,19366,21318,24407,19423,21815
1819,총범죄,지능범죄,소계,광주,7973,8980,9812,9628,9724,8896,8401,8977,9651,10777,9162,10463
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1898,총범죄,지능범죄,소계,통영시,0,0,0,0,704,858,841,1567,1344,1179,820,1367
1899,총범죄,지능범죄,소계,서귀포시,0,0,0,0,988,955,958,1196,1404,1451,1296,1875
1900,총범죄,지능범죄,소계,제주시,3612,4211,3935,3741,3403,3108,3928,4534,3948,4719,4215,4879
1901,총범죄,지능범죄,소계,기타도시,20541,22666,24765,23603,2495,2055,1831,4557,8069,13860,14713,27106


In [40]:
mastermind_dosi_df["위도"] = korea_city_lat
mastermind_dosi_df["경도"] = korea_city_lon
mastermind_dosi_df.head()

/tmp/ipykernel_11040/2692303829.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_11040/2692303829.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,대,중,소,발생장소,2011년,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,2022년,위도,경도
1815,총범죄,지능범죄,소계,서울,71386,72333,76756,72052,72851,72137,70294,74668,78857,82443,64838,72239,37.5665,126.9780
1816,총범죄,지능범죄,소계,부산,28310,25502,28191,24263,25247,25052,23726,27672,31357,36784,30249,32816,35.1796,129.0756
1817,총범죄,지능범죄,소계,대구,12957,12859,13639,12868,14195,13227,13206,15021,18218,19812,17979,18192,35.8714,128.6014
1818,총범죄,지능범죄,소계,인천,12511,14059,18957,16580,16938,17248,16147,19366,21318,24407,19423,21815,37.4563,126.7052
1819,총범죄,지능범죄,소계,광주,7973,8980,9812,9628,9724,8896,8401,8977,9651,10777,9162,10463,35.1595,126.8526


In [41]:
fig = go.Figure()

for  year in mastermind_dosi_df.columns[4:16]:
    fig.add_trace(go.Scattergeo(
        locationmode='ISO-3',
        lon=mastermind_dosi_df["경도"],
        lat=mastermind_dosi_df["위도"],
        text=('발생 장소: ' + mastermind_dosi_df["발생장소"] + ' / 발생 건수: ' + mastermind_dosi_df[year].astype(str)).values,
        name= "{0} 지능범죄 발생건수".format(year),
        marker=dict(
            size=(mastermind_dosi_df[year] / 500).values,
            color="lightseagreen",
            line_color = 'rgb(40,40,40)',
            line_width = 0.5,
        ),
        mode='markers',
        visible=False
    ))

fig.data[0].visible = True

steps = []
years  = mastermind_dosi_df.columns[4:16]

for i in range(len(set(years))):
    step = dict(
        method = 'update',
        label=str(sorted(set(years))[i]),
        args=[{'visible':[False]*len(fig.data)},
              {'title': '{} 지능범죄 지도 시각화'.format(sorted(set(years))[i])}],
    )

    step['args'][0]['visible'][i] = True
    steps.append(step)


fig.update_layout( 
    title_text='<b>연도별 발생지역의 지능범죄 지도 시각화</b>',
    title_x=0.5, 
    title_font_size=22,
    geo=dict(
        scope='asia',
        resolution=50,
        showland=True,
        landcolor="rgb(240, 240, 240)",
        showcountries=True,
        countrycolor="rgb(170, 170, 170)",
        lataxis=dict(
            range=[33, 39]
        ),
        lonaxis=dict(
            range=[124, 131]
        ),
        projection=dict(
            type='mercator',
            scale=0.9 # Adjusting scale to make the map larger
        ),
        center=dict(
            lat=36.5,  # Centering map on South Korea
            lon=127.5
        )
    ),
    sliders=[{
        'active':0,
        'steps':steps,
    }],
    width=900,  # Adjust width of the plot
    height=1000  # Adjust height of the plot
)
fig.show()

fig.write_html("intelligent_dosi_map.html")

In [42]:
violent_dosi_df = crime_dosi_df[(crime_dosi_df["중"] == "폭력범죄") & (crime_dosi_df["소"] == "소계") ]
violent_dosi_df

,대,중,소,발생장소,2011년,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,2022년
1023,총범죄,폭력범죄,소계,서울,72056,70668,65433,65316,65247,64601,59401,56090,54688,48308,41748,46710
1024,총범죄,폭력범죄,소계,부산,20669,22516,21109,21142,20928,20444,20497,19907,20668,19086,16129,17242
1025,총범죄,폭력범죄,소계,대구,17657,16897,14216,13738,14708,14557,13690,13374,13387,12278,11002,11360
1026,총범죄,폭력범죄,소계,인천,16167,17350,16569,16791,18054,19881,19410,19350,19663,17798,14983,15295
1027,총범죄,폭력범죄,소계,광주,11279,11605,10211,10185,9602,8610,8367,8266,8683,7360,6617,6302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1106,총범죄,폭력범죄,소계,통영시,0,0,0,0,1006,985,877,905,869,931,736,752
1107,총범죄,폭력범죄,소계,서귀포시,0,0,0,0,1382,1632,1450,1468,1304,1344,1349,1409
1108,총범죄,폭력범죄,소계,제주시,5640,4685,5652,5172,4136,4471,3853,3924,3843,3997,3763,3978
1109,총범죄,폭력범죄,소계,기타도시,21568,22543,22011,22009,825,493,124,180,289,365,378,638


In [43]:
violent_dosi_df["위도"] = korea_city_lat
violent_dosi_df["경도"] = korea_city_lon
violent_dosi_df.head()

/tmp/ipykernel_11040/1973334091.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_11040/1973334091.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,대,중,소,발생장소,2011년,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,2022년,위도,경도
1023,총범죄,폭력범죄,소계,서울,72056,70668,65433,65316,65247,64601,59401,56090,54688,48308,41748,46710,37.5665,126.9780
1024,총범죄,폭력범죄,소계,부산,20669,22516,21109,21142,20928,20444,20497,19907,20668,19086,16129,17242,35.1796,129.0756
1025,총범죄,폭력범죄,소계,대구,17657,16897,14216,13738,14708,14557,13690,13374,13387,12278,11002,11360,35.8714,128.6014
1026,총범죄,폭력범죄,소계,인천,16167,17350,16569,16791,18054,19881,19410,19350,19663,17798,14983,15295,37.4563,126.7052
1027,총범죄,폭력범죄,소계,광주,11279,11605,10211,10185,9602,8610,8367,8266,8683,7360,6617,6302,35.1595,126.8526


In [44]:
fig = go.Figure()

for  year in violent_dosi_df.columns[4:16]:
    fig.add_trace(go.Scattergeo(
        locationmode='ISO-3',
        lon=violent_dosi_df["경도"],
        lat=violent_dosi_df["위도"],
        text=('발생 장소: ' + violent_dosi_df["발생장소"] + ' / 발생 건수: ' + violent_dosi_df[year].astype(str)).values,
        name= "{0} 폭력범죄 발생건수".format(year),
        marker=dict(
            size=(violent_dosi_df[year] / 500).values,
            color="orange",
            line_color = 'rgb(40,40,40)',
            line_width = 0.5,
        ),
        mode='markers',
        visible=False
    ))

fig.data[0].visible = True

steps = []
years  = violent_dosi_df.columns[4:16]

for i in range(len(set(years))):
    step = dict(
        method = 'update',
        label=str(sorted(set(years))[i]),
        args=[{'visible':[False]*len(fig.data)},
              {'title': '{} 폭력범죄 지도 시각화'.format(sorted(set(years))[i])}],
    )

    step['args'][0]['visible'][i] = True
    steps.append(step)


fig.update_layout( 
    title_text='<b>연도별 발생지역의 폭력범죄 지도 시각화</b>',
    title_x=0.5, 
    title_font_size=22,
    geo=dict(
        scope='asia',
        resolution=50,
        showland=True,
        landcolor="rgb(240, 240, 240)",
        showcountries=True,
        countrycolor="rgb(170, 170, 170)",
        lataxis=dict(
            range=[33, 39]
        ),
        lonaxis=dict(
            range=[124, 131]
        ),
        projection=dict(
            type='mercator',
            scale=0.9  # Adjusting scale to make the map larger
        ),
        center=dict(
            lat=36.5,  # Centering map on South Korea
            lon=127.5
        )
    ),
    sliders=[{
        'active':0,
        'steps':steps,
    }],
    width=900,  # Adjust width of the plot
    height=1000  # Adjust height of the plot
)
fig.show()

fig.write_html("violent_dosi_map.html")

In [45]:
vehicle_dosi_df = crime_dosi_df[(crime_dosi_df["중"] == "교통범죄") & (crime_dosi_df["소"] == "소계") ]
vehicle_dosi_df

,대,중,소,발생장소,2011년,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,2022년
3311,총범죄,교통범죄,소계,서울,87467,79627,86560,76497,74958,74270,67430,61132,55273,50895,43206,32013
3312,총범죄,교통범죄,소계,부산,32131,32758,31638,36035,35841,32944,28061,22894,20433,18201,16160,12176
3313,총범죄,교통범죄,소계,대구,34209,36180,35431,38676,39725,31682,25380,22200,21012,19005,16619,12610
3314,총범죄,교통범죄,소계,인천,28086,30491,30232,32322,31163,30972,24507,18173,17749,16771,14459,11926
3315,총범죄,교통범죄,소계,광주,21878,24387,26251,24008,23412,22137,18734,13779,12747,12432,11121,7827
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3394,총범죄,교통범죄,소계,통영시,0,0,0,0,2936,2430,2031,1350,1262,1245,1019,825
3395,총범죄,교통범죄,소계,서귀포시,0,0,0,0,3581,4060,3713,2378,1895,1904,1916,1382
3396,총범죄,교통범죄,소계,제주시,7422,9328,11848,12953,10857,11642,9936,6417,5147,4632,4266,3052
3397,총범죄,교통범죄,소계,기타도시,45967,47685,53138,55076,2325,1287,239,319,184,129,204,323


In [46]:
vehicle_dosi_df["위도"] = korea_city_lat
vehicle_dosi_df["경도"] = korea_city_lon
vehicle_dosi_df.head()

/tmp/ipykernel_11040/424551779.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_11040/424551779.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,대,중,소,발생장소,2011년,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,2022년,위도,경도
3311,총범죄,교통범죄,소계,서울,87467,79627,86560,76497,74958,74270,67430,61132,55273,50895,43206,32013,37.5665,126.9780
3312,총범죄,교통범죄,소계,부산,32131,32758,31638,36035,35841,32944,28061,22894,20433,18201,16160,12176,35.1796,129.0756
3313,총범죄,교통범죄,소계,대구,34209,36180,35431,38676,39725,31682,25380,22200,21012,19005,16619,12610,35.8714,128.6014
3314,총범죄,교통범죄,소계,인천,28086,30491,30232,32322,31163,30972,24507,18173,17749,16771,14459,11926,37.4563,126.7052
3315,총범죄,교통범죄,소계,광주,21878,24387,26251,24008,23412,22137,18734,13779,12747,12432,11121,7827,35.1595,126.8526


In [47]:
fig = go.Figure()

for  year in vehicle_dosi_df.columns[4:16]:
    fig.add_trace(go.Scattergeo(
        locationmode='ISO-3',
        lon=vehicle_dosi_df["경도"],
        lat=vehicle_dosi_df["위도"],
        text=('발생 장소: ' + vehicle_dosi_df["발생장소"] + ' / 발생 건수: ' + vehicle_dosi_df[year].astype(str)).values,
        name= "{0} 교통범죄 발생건수".format(year),
        marker=dict(
            size=(vehicle_dosi_df[year] / 580).values,
            color="forestgreen",
            line_color = 'rgb(40,40,40)',
            line_width = 0.5,
        ),
        mode='markers',
        visible=False
    ))

fig.data[0].visible = True

steps = []
years  = vehicle_dosi_df.columns[4:16]

for i in range(len(set(years))):
    step = dict(
        method = 'update',
        label=str(sorted(set(years))[i]),
        args=[{'visible':[False]*len(fig.data)},
              {'title': '{} 교통범죄 지도 시각화'.format(sorted(set(years))[i])}],
    )

    step['args'][0]['visible'][i] = True
    steps.append(step)


fig.update_layout( 
    title_text='<b>연도별 발생지역의 교통범죄 지도 시각화</b>',
    title_x=0.5, 
    title_font_size=22,
    geo=dict(
        scope='asia',
        resolution=50,
        showland=True,
        landcolor="rgb(240, 240, 240)",
        showcountries=True,
        countrycolor="rgb(170, 170, 170)",
        lataxis=dict(
            range=[33, 39]
        ),
        lonaxis=dict(
            range=[124, 131]
        ),
        projection=dict(
            type='mercator',
            scale=0.9  # Adjusting scale to make the map larger
        ),
        center=dict(
            lat=36.5,  # Centering map on South Korea
            lon=127.5
        )
    ),
    sliders=[{
        'active':0,
        'steps':steps,
    }],
    width=900,  # Adjust width of the plot
    height=1000  # Adjust height of the plot
)
fig.show()

fig.write_html("traffic_dosi_map.html")